##Load up the libraries

In [ ]:
!pip install transformers
!pip install torchmetrics
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.2/869.2 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which 

In [ ]:
import pandas as pd
import numpy as np
import re
from transformers import AutoTokenizer, BertModel, BertForSequenceClassification, BertConfig
from tqdm import tqdm
import torch
import pickle
from torch.utils.data import TensorDataset, DataLoader
from torchmetrics import Accuracy
from datasets import load_dataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## A function to pre-process each line

In [ ]:
def preprocess(x):
    x = re.sub('<.*?>', ' ', x)
    x = re.sub('http\S+', ' ', x)
    x = re.sub('\s+', ' ', x)
    return x.lower().strip()

## Helper functions to save and load pickle files

In [ ]:
def save_pickle_file(object, file_name):
    fp = open(file_name, "wb")
    pickle.dump(object, fp)
    fp.close()


def load_pickle_file(file_name):
    fp = open(file_name, "rb")
    data = pickle.load(fp)
    fp.close()
    return data

## This function converts input dataframe to transformer usable format

In [ ]:
def pipeline(dataframe):
    # Pre-process the sentences
    dataframe['text'] = dataframe['text'].apply(lambda x: preprocess(x))

    # Pre-pend CLS token to each sentence
    sentences = ["[CLS] " + s for s in dataframe.text.values]

    # Extract labels
    labels = dataframe.label.values

    # Tokenize each
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)
    tokenized = [tokenizer.tokenize(s) for s in tqdm(sentences)]

    # Append the SEP token and also set a threshold for the number of tokens in a sentence
    MAX_LEN_TRAIN, MAX_LEN_TEST = 140, 140
    tokenized = [t[:(MAX_LEN_TRAIN-1)]+['SEP'] for t in tokenized]

    # Generate IDs of each token and add padding to sentences smaller than given threshold
    ids = [tokenizer.convert_tokens_to_ids(t) for t in tqdm(tokenized)]
    ids = np.array([np.pad(i, (0, MAX_LEN_TRAIN-len(i)), mode='constant') for i in ids])

    # Also generate Attention masks. An attention mask is a binary tensor
    # that indicates the position of padded indices so that the model does not attend to them
    amasks = np.asarray([[float(i>0) for i in seq] for seq in tqdm(ids)])

    # Tokenize each
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)
    tokenized = [tokenizer.tokenize(s) for s in tqdm(sentences)]

    # Append the SEP token and also set a threshold for the number of tokens in a sentence
    MAX_LEN_TRAIN, MAX_LEN_TEST = 140, 140
    tokenized = [t[:(MAX_LEN_TRAIN-1)]+['SEP'] for t in tokenized]

    # Generate IDs of each token and add padding to sentences smaller than given threshold
    ids = [tokenizer.convert_tokens_to_ids(t) for t in tqdm(tokenized)]


    return torch.tensor(ids), torch.tensor(labels), torch.tensor(amasks)

## Load the training and validation datasets

In [ ]:
dataset = load_dataset('imdb')
df_train = dataset['train'].to_pandas()
display(df_train.head())

df_val = dataset['test'].to_pandas()
display(df_val.head())

## Clean the data and store in BERT usable format

In [ ]:
ids_train, labels_train, amasks_train = pipeline(df_train)
ids_val, labels_val, amasks_val = pipeline(df_val)

print(ids_train.shape, amasks_train.shape, labels_train.shape)
print(ids_val.shape, amasks_val.shape, labels_val.shape)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
100%|██████████| 25000/25000 [00:01<00:00, 16205.44it/s]


ValueError: expected sequence of length 140 at dim 1 (got 109)

## Generate the data loaders

In [ ]:
train_set = TensorDataset(ids_train, amasks_train, labels_train)
train_dataloader = DataLoader(train_set, batch_size=32, shuffle=True)

val_set = TensorDataset(ids_val, amasks_val, labels_val)
val_dataloader = DataLoader(val_set, batch_size=32, shuffle=False)

## Now Create the model

In [ ]:
# If you load a pre-trained model and train it using a small learning rate
# then it is known as fine-tuning.
# If you provide a config file then the BERT model is loaded without the
# pretrained weights. Training this model is known as training from scratch

# Loading your model this way loads a network without the pre-trained weights
# config = BertConfig.from_pretrained("bert-base-uncased")
# print(config)
# model = BertForSequenceClassification(config)

# Loading your model this way loads the pre-trained network
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")

print(model)
# You can also replace the last classification layer with your own layer
# Based on your classification needs
# model.classifier = torch.nn.Linear(768, 10)
# print("\n\nNew BertModel:\n", model)
model = model.to(device)

# You can also freeze some of the layers in Bert
# If you freeze all the layers except the classification layer
# and train the model, then this is known as transfer learning
for idx, (name, param) in enumerate(model.named_parameters()):
    # print(idx, name)
    if "classifier" in name or "bert.encoder.layer.9" in name or "bert.encoder.layer.10" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

total_params = 0
for param in model.parameters():
    if param.requires_grad:
        total_params+= param.numel()
#print(total_params)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

## Train and Test the model

In [ ]:
epochs = 10
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0003, eps=1e-8) #recommended for transformer architecture
criterion = torch.nn.CrossEntropyLoss()
train_acc, val_acc = Accuracy(task="binary", num_classes=2).to(device), Accuracy(task="binary", num_classes=2).to(device)

# We will train the model for the specified number of epochs
for epoch in range(epochs):
    train_loss, val_loss = list(), list()
    print("\n\nEpoch:", epoch, "\n-----------------------\n")
    # Make sure model is in training mode
    model.train()
    # For each batch of data
    for idx, (x_ids, x_masks, x_labels) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        # Clear previous gradients
        optimizer.zero_grad()
        # Move the batch to the GPU
        x_ids, x_masks, x_labels = x_ids.to(device), x_masks.to(device), x_labels.to(device)
        # Perform predictions
        preds = model(x_ids, attention_mask=x_masks)
        # Save the current iteration's accuracy
        train_acc.update(torch.argmax(preds.logits, dim=1), x_labels)
        # Get the loss
        loss = criterion(preds.logits, x_labels)
        train_loss.append(loss.item())
        # Calculate the gradients
        loss.backward()
        # Update the parameters with the calculated gradients
        optimizer.step()

    # After each epoch, test the model
    model.eval()
    for idx, (x_ids, x_masks, x_labels) in tqdm(enumerate(val_dataloader), total=len(val_dataloader)):
        x_ids, x_masks, x_labels = x_ids.to(device), x_masks.to(device), x_labels.to(device)
        preds = model(x_ids, attention_mask=x_masks)
        loss = criterion(preds.logits, x_labels)
        val_loss.append(loss.item())
        val_acc.update(torch.argmax(preds.logits, dim=1), x_labels)

    # Finally print out the average train and val losses
    print("Train Loss =", sum(train_loss)/len(train_loss), "\tVal Loss =", sum(val_loss)/len(val_loss))
    # As well as the train and val accuracies
    print("Train Acc =", train_acc.compute().item(), "\tVal Acc =", val_acc.compute().item())



Epoch: 0 
-----------------------



100%|██████████| 782/782 [03:32<00:00,  3.68it/s]


Train Loss = 0.40798996611857963 	Val Loss = 0.32611097621581403
Train Acc = 0.8130800127983093 	Val Acc = 0.8652399778366089


Epoch: 1 
-----------------------



100%|██████████| 782/782 [03:31<00:00,  3.69it/s]


Train Loss = 0.32807882086319085 	Val Loss = 0.3131021815768974
Train Acc = 0.835099995136261 	Val Acc = 0.8663399815559387


Epoch: 2 
-----------------------



...

Train Loss = 0.2455117650368177 	Val Loss = 0.3179241855476347
Train Acc = 0.8688349723815918 	Val Acc = 0.8671900033950806


Epoch: 8 
-----------------------



100%|██████████| 782/782 [03:30<00:00,  3.71it/s]


Train Loss = 0.237624605436383 	Val Loss = 0.31924501861519444
Train Acc = 0.8726222515106201 	Val Acc = 0.8678088784217834


Epoch: 9 
-----------------------



100%|██████████| 782/782 [03:31<00:00,  3.70it/s]

Train Loss = 0.23459976528059034 	Val Loss = 0.3054336016983404
Train Acc = 0.8756240010261536 	Val Acc = 0.8684520125389099
